In [1]:
# %pip install torch
# %pip install torchprofile
# %pip install thop
# %pip install ptflops
# %pip install calflops
# %pip install transformers

In [2]:
import torch 
import torch.nn as nn
from torchprofile import profile_macs
from thop import profile
from ptflops import get_model_complexity_info
from torchinfo  import summary
from calflops import calculate_flops

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class My_model(nn.Module):

    def __init__(self,in_ch,out_ch,ker,st):
        super(My_model,self).__init__()
        self.cnn = nn.Conv1d(in_ch,out_ch,kernel_size=ker,stride=st)

    def forward(self,x):
        
        print("Input ", x.shape)
        x = self.cnn(x)
        print("Output ",x.shape)
        return x

in_ch = 512
out_ch = 256
ker = 3
st = 1
time = 1000

dummy_input = torch.rand(1,in_ch,time)

model = My_model(in_ch,out_ch,ker,st)
model.eval()
print(summary(model, input_data= dummy_input, device='cpu', depth=5))

macs = profile_macs(model, dummy_input)

hand_mac = (in_ch*out_ch*ker*((time-ker)//st+1))

print(f'MACs: {macs}')
print(f"Hand Macs: {hand_mac}")


# in_ch  2
# out_ch 32
# ker_f  5
# ker_t  2
# Out_T  121
# Out_F  127       
# Mac = in_ch*out_ch*ker_t*ker_f*out_t*out_f


Input  torch.Size([1, 512, 1000])
Output  torch.Size([1, 256, 998])
Layer (type:depth-idx)                   Output Shape              Param #
My_model                                 [1, 256, 998]             --
├─Conv1d: 1-1                            [1, 256, 998]             393,472
Total params: 393,472
Trainable params: 393,472
Non-trainable params: 0
Total mult-adds (M): 392.69
Input size (MB): 2.05
Forward/backward pass size (MB): 2.04
Params size (MB): 1.57
Estimated Total Size (MB): 5.67
Input  torch.Size([1, 512, 1000])
Output  torch.Size([1, 256, 998])
MACs: 392429568
Hand Macs: 392429568


In [29]:
class My_model(nn.Module):

    def __init__(self,in_ch,out_ch,ker,st):
        super(My_model,self).__init__()
        self.cnn = nn.Conv2d(in_ch,out_ch,kernel_size=ker,stride=st)

    def forward(self,x):
        
        print("Input ", x.shape)
        x = self.cnn(x)
        print("Output ",x.shape)
        return x

in_ch = 2
out_ch = 512
ker = (5,3)
st = (2,1)
time = 122
freq = 257

dummy_input = torch.rand(1,in_ch,freq,time)

model = My_model(in_ch,out_ch,ker,st)
model.eval()
print(summary(model, input_data= dummy_input, device='cpu', depth=5))

macs = profile_macs(model, dummy_input)

t_out = (time-ker[1])//st[1]+1
f_out = (freq-ker[0])//st[0]+1

hand_mac = (in_ch*out_ch*ker[0]*ker[1]*t_out*f_out)

print(f'MACs: {macs}')
print(f"Hand Macs: {hand_mac}")


# in_ch  2
# out_ch 32
# ker_f  5
# ker_t  2
# Out_T  121
# Out_F  127       
# Mac = in_ch*out_ch*ker_t*ker_f*out_t*out_f


Input  torch.Size([1, 2, 257, 122])
Output  torch.Size([1, 512, 127, 120])
Layer (type:depth-idx)                   Output Shape              Param #
My_model                                 [1, 512, 127, 120]        --
├─Conv2d: 1-1                            [1, 512, 127, 120]        15,872
Total params: 15,872
Trainable params: 15,872
Non-trainable params: 0
Total mult-adds (M): 241.89
Input size (MB): 0.25
Forward/backward pass size (MB): 62.42
Params size (MB): 0.06
Estimated Total Size (MB): 62.74
Input  torch.Size([1, 2, 257, 122])
Output  torch.Size([1, 512, 127, 120])
MACs: 234086400
Hand Macs: 234086400


In [22]:
class My_model(nn.Module):

    def __init__(self,feat,hidden_size,dir):
        super(My_model,self).__init__()
        self.gru = nn.GRU(feat,hidden_size,dir,batch_first=True)

    def forward(self,x):
        
        B,H,W = x.shape
        print("Input ", x.shape) 

        x = x.permute(0, 2, 1) # (B, W, C, H)
        print("Permute ",x.shape)

        output, hn = self.gru(x)

        print("Output ",output.shape)
        print("hn ",hn.shape)

        return output,hn

time = 1
freq = 257
batch = 1
hidden_size = 64
dir = 1 # 2 for bidirectional

dummy_input = torch.rand(1,257,1)
model = My_model(freq,hidden_size,dir)
model.eval()
print(summary(model, input_data = dummy_input, device='cpu', depth=5))

total_params = 0

for name, p in model.named_parameters():
    if p.requires_grad:
        num_params = p.numel()
        total_params += num_params
        print(f'Layer: {name}, Parameters: {num_params}, shape {p.shape}')

print(f'Total parameters: {total_params}')

# macs = profile_macs(model, dummy_input)
# print(f'profile_macs MACs: {macs}')


macs, params = profile(model, inputs=(dummy_input,))
print(f'profile params: {params}')       
print(f'profile MACs: {macs}')

parms_hand = 3*(hidden_size**2 + hidden_size*freq + 2*hidden_size)*dir 

mac_hand = 3*time*(hidden_size**2 + hidden_size*freq + 2*hidden_size ) + 3*hidden_size + freq

print(f'MACs by hand: {mac_hand}')


# flops, macs, params = calculate_flops(model=model, 
#                                       input_shape=input_shape,
#                                       output_as_string=False,
#                                       output_precision=4)
# print(" FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))

Input  torch.Size([1, 257, 1])
Permute  torch.Size([1, 1, 257])
Output  torch.Size([1, 1, 64])
hn  torch.Size([1, 1, 64])
Layer (type:depth-idx)                   Output Shape              Param #
My_model                                 [1, 1, 64]                --
├─GRU: 1-1                               [1, 1, 64]                62,016
Total params: 62,016
Trainable params: 62,016
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.25
Estimated Total Size (MB): 0.25
Layer: gru.weight_ih_l0, Parameters: 49344, shape torch.Size([192, 257])
Layer: gru.weight_hh_l0, Parameters: 12288, shape torch.Size([192, 64])
Layer: gru.bias_ih_l0, Parameters: 192, shape torch.Size([192])
Layer: gru.bias_hh_l0, Parameters: 192, shape torch.Size([192])
Total parameters: 62016
[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
Input  torch.Size([1, 257, 1])
Permute  torch.Size([1, 1, 257])
Output  torch.Size(

In [10]:
class My_model(nn.Module):

    def __init__(self,axis):
        super(My_model,self).__init__()

        self.LN = nn.LayerNorm(axis)

    def forward(self,x):

        B,C,H,W = x.shape
        print("Input Shape",x.shape)
        x = x.reshape(B*W,H,C)
        print("Reshape ",x.shape)

        x = self.LN(x)

        print("Layer norm out",x.shape)
        x = x.reshape(B,C,H,W)
        print("Reshape out",x.shape)

        return x

batch = 1
in_ch = 2
freq = 257
time = 122

dummy_input = torch.rand(batch,in_ch,freq,time)
model = My_model([freq,in_ch])
model.eval()
print(summary(model, input_data= dummy_input, device='cpu', depth=5))

total_params = 0

print("Weights each layer")
for name, p in model.named_parameters():
    if p.requires_grad:
        num_params = p.numel()
        total_params += num_params
        print(f'Layer: {name}, Parameters: {num_params}')

print(f'Total parameters: {total_params}')

macs = profile_macs(model, dummy_input)
print(f'profile_macs MACs: {macs}')

macs_hand = (batch*time)*freq*in_ch+1

print(f'MACs by Hand: {macs_hand}')


#Mac = Batch*Feat*Time_frames + 1


Input Shape torch.Size([1, 2, 257, 122])
Reshape  torch.Size([122, 257, 2])
Layer norm out torch.Size([122, 257, 2])
Reshape out torch.Size([1, 2, 257, 122])
Layer (type:depth-idx)                   Output Shape              Param #
My_model                                 [1, 2, 257, 122]          --
├─LayerNorm: 1-1                         [122, 257, 2]             1,028
Total params: 1,028
Trainable params: 1,028
Non-trainable params: 0
Total mult-adds (M): 0.13
Input size (MB): 0.25
Forward/backward pass size (MB): 0.50
Params size (MB): 0.00
Estimated Total Size (MB): 0.76
Weights each layer
Layer: LN.weight, Parameters: 514
Layer: LN.bias, Parameters: 514
Total parameters: 1028
Input Shape torch.Size([1, 2, 257, 122])
Reshape  torch.Size([122, 257, 2])
Layer norm out torch.Size([122, 257, 2])
Reshape out torch.Size([1, 2, 257, 122])
 profile_macs MACs: 62709
MACs by Hand: 62709


/home/user/.local/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [8]:
import torch
import torch.nn as nn
from torchprofile import profile_macs

class InstantLayerNormalization(nn.Module):
    '''
    Class implementing instant layer normalization. It can also be called 
    channel-wise layer normalization and was proposed by 
    Luo & Mesgarani (https://arxiv.org/abs/1809.07454v2) 
    '''

    def __init__(self, num_channels, epsilon=1e-7):
        '''
        Constructor
        '''
        super(InstantLayerNormalization, self).__init__()
        self.epsilon = epsilon
        
        # Initialize gamma and beta
        self.gamma = nn.Parameter(torch.ones(num_channels))
        self.beta = nn.Parameter(torch.zeros(num_channels))

    def forward(self, inputs):
        '''
        Method to call the Layer. All processing is done here.
        '''
        # Calculate mean of each frame
        mean = inputs.mean(dim=-1, keepdim=True)
        
        # Calculate variance of each frame
        variance = ((inputs - mean) ** 2).mean(dim=-1, keepdim=True)
        
        # Calculate standard deviation
        std = torch.sqrt(variance + self.epsilon)
        
        # Normalize each frame independently
        outputs = (inputs - mean) / std
        
        # Scale with gamma
        outputs = self.gamma * outputs
        
        # Add the bias beta
        outputs = self.beta + outputs 
        
        return outputs

batch = 1
time = 122
in_ch = 256   # Example: can be adjusted based on your data
model = InstantLayerNormalization(in_ch)

# Generate some example input data
# Let's say we have a batch size of 4, sequence length of 10, and 16 channels


# Create random input tensor with shape (batch_size, sequence_length, num_channels)
dummy_input = torch.randn(batch, time, in_ch)

# Perform inference
output_data = model(dummy_input)

# Print the output shape
print("Input shape:", dummy_input.shape)
print("Output shape:", output_data.shape)


print(summary(model, input_data= dummy_input, device='cpu', depth=5))

total_params = 0

for name, p in model.named_parameters():
    if p.requires_grad:
        num_params = p.numel()
        total_params += num_params
        print(f'Layer: {name}, Parameters: {num_params}, shape {p.shape}')

print(f'Total parameters: {total_params}')

macs = profile_macs(model, dummy_input)
print(f'profile_macs MACs: {macs}')

macs_hand = (batch*time*in_ch)+(in_ch)

print(f'MACs by Hand: {macs_hand}')


# macs, params = profile(model, inputs=(dummy_input,))
# print(f'profile MACs: {macs}')
# print(f'profile Parms: {params}')


Input shape: torch.Size([1, 122, 256])
Output shape: torch.Size([1, 122, 256])
Layer (type:depth-idx)                   Output Shape              Param #
InstantLayerNormalization                [1, 122, 256]             512
Total params: 512
Trainable params: 512
Non-trainable params: 0
Total mult-adds (M): 0
Input size (MB): 0.12
Forward/backward pass size (MB): 0.25
Params size (MB): 0.00
Estimated Total Size (MB): 0.38
Layer: gamma, Parameters: 256, shape torch.Size([256])
Layer: beta, Parameters: 256, shape torch.Size([256])
Total parameters: 512
profile_macs MACs: 31476
MACs by Hand: 31488


/home/user/.local/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pow". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/user/.local/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::sqrt". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [19]:
## For pretrained model

import torch
from torchvision.models import resnet50

from thop import profile

model = resnet50()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input,))
print(f'MACs: {macs}')
print(f'Parms: {params}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 4133742592.0
Parms: 25557032.0


In [9]:
import torch
import torch.nn as nn
from thop import profile, clever_format

# Define a simple neural network model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, dilation = 2,groups = 1,stride=1, padding=0)
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # self.fc1 = nn.Linear(32 * 7 * 7, 128)
        # self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        # x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        # # x = nn.functional.relu(self.conv2(x))
        # # x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        # # x = x.view(x.size(0), -1)  # Flatten
        # # x = nn.functional.relu(self.fc1(x))
        # # x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleNet()

# Create a dummy input tensor
input_tensor = torch.randn(1, 4, 28, 28)  # Batch size of 1, 1 channel, 28x28 image

# Calculate MACs using thop
macs, params = profile(model, inputs=(input_tensor,))

print(f"profile Total MACs: {macs}")
print(f"profile Total parameters: {params}")

# Print the results in term of K 
macs, params = clever_format([macs, params], "%.3f")
print(f"clever_format,Total MACs: {macs}")
print(f"clever_format Total parameters: {params}")


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
torch.Size([1, 16, 24, 24])
profile Total MACs: 331776.0
profile Total parameters: 592.0
clever_format,Total MACs: 331.776K
clever_format Total parameters: 592.000B
